In [ ]:
import os
import MDAnalysis as mda
import numpy as np
import seaborn
from MDAnalysis.analysis.distances import distance_array
from matplotlib import pyplot as plt
from numpy import vectorize

class ContactMap:

    def __init__(self, top_path: str, trj_path=None):
        self.ag = None
        if not trj_path:
            self.u = mda.Universe(top_path)
        else:
            self.u = mda.Universe(top_path, trj_path)

        self.yticks = None
        self.xticks = None

    @staticmethod
    @vectorize
    def binarize(x, cut_off: float):
        return 1 if x < cut_off else 0

    def _pairwise_contacts(self, a, b, cutoff):
        distances = distance_array(a, b)
        return self.binarize(distances, cutoff)

    def calculate_contacts(self, selection_1: str, selection_2: str, cutoff=5, start=0, end=None, skip=1):
        ag_1 = self.u.select_atoms(selection_1)
        ag_2 = self.u.select_atoms(selection_2)
        self.xticks = ag_1[ag_1.resids == ag_1.resids[0]].names
        self.yticks = ag_1[ag_1.resids == ag_1.resids[0]].names

        # Number of atoms in each residue
        res_length_1 = len(ag_1[ag_1.resids == ag_1.resids[0]])
        res_length_2 = len(ag_2[ag_2.resids == ag_2.resids[0]])

        # Number of molecules
        n_mol_1 = len(set(ag_1.resids))
        n_mol_2 = len(set(ag_2.resids))

        trj_length = len(self.u.trajectory)

        if end is None:
            end = trj_length

        results = []

        for i, _ in enumerate(self.u.trajectory[start:end:skip]):
            if i % 10 == 0:
                print('Finished frame:', i)
            # Set positions and reshape
            pos_1 = ag_1.positions.reshape((n_mol_1, res_length_1, 3))
            pos_2 = ag_2.positions.reshape((n_mol_2, res_length_2, 3))
            for p_1 in pos_1:
                for p_2 in pos_2:
                    results.append(self._pairwise_contacts(p_1, p_2, cutoff=cutoff))

        results = np.array(results)
        trj_mean = results.mean(axis=0)

        return trj_mean / trj_mean.max()

    def save_contact_map(self, data, output_file="contact_map.dat"):
        """
        Save the contact map to a .dat file in the format:
        Atom1 Atom2 InteractionStrength
        """
        with open(output_file, "w") as f:
            for i, atom1 in enumerate(self.xticks):
                for j, atom2 in enumerate(self.yticks):
                    f.write(f"{atom1} {atom2} {data[i, j]:.4f}\n")
        print(f"Contact map saved to {output_file}")


if __name__ == '__main__':
    BASE_PATH = r"project3/"
    SYSTEM_NAME = "asp_trx"
    top = os.path.join(BASE_PATH, SYSTEM_NAME, "asp900ns.tpr")
    trj = os.path.join(BASE_PATH, SYSTEM_NAME, "merged_trajectory.xtc")
    
    m = ContactMap(top, trj)
    #print(m.u.select_atoms('all').resnames)
    res = m.calculate_contacts(selection_1='resname ASP and not type H*',
                               selection_2='resname ASP and not type H*',
                               start=900, end=1000
                               )
    
    # Save the contact map to a .dat file
    res = res.T  # Fix the transpose issue
    m.save_contact_map(res, output_file="ASP_ASP_contact_map.dat")
    
    # Generate and save the heatmap
    seaborn.heatmap(res, cmap='coolwarm', xticklabels=m.xticks, yticklabels=m.yticks)
    plt.savefig('ASP-ASP_MH1.jpg', dpi=300)
    plt.show()


In [ ]:
import os
import MDAnalysis as mda
import numpy as np
import seaborn
from MDAnalysis.analysis.distances import distance_array
from matplotlib import pyplot as plt
from numpy import vectorize

class ContactMap:
    def __init__(self, top_path: str, trj_path=None):
        self.ag = None
        if not trj_path:
            self.u = mda.Universe(top_path)
        else:
            self.u = mda.Universe(top_path, trj_path)
        self.yticks = None
        self.xticks = None
    
    @staticmethod
    @vectorize
    def binarize(x, cut_off: float):
        return 1 if x < cut_off else 0
    
    def _pairwise_contacts(self, a, b, cutoff):
        distances = distance_array(a, b)
        return self.binarize(distances, cutoff)
    
    def calculate_contacts(self, selection_1: str, selection_2: str, cutoff=5, start=0, end=None, skip=1):
        ag_1 = self.u.select_atoms(selection_1)
        ag_2 = self.u.select_atoms(selection_2)
        self.xticks = ag_1[ag_1.resids == ag_1.resids[0]].names
        self.yticks = ag_2[ag_2.resids == ag_2.resids[0]].names
        
        # Number of atoms in each residue
        res_length_1 = len(ag_1[ag_1.resids == ag_1.resids[0]])
        res_length_2 = len(ag_2[ag_2.resids == ag_2.resids[0]])
        
        # Number of molecules
        n_mol_1 = len(set(ag_1.resids))
        n_mol_2 = len(set(ag_2.resids))
        trj_length = len(self.u.trajectory)
        
        print("res_length_1 (TRITO):", res_length_1)
        print("res_length_2 (ASP):", res_length_2)
        print("n_mol_1 (TRITO molecules):", n_mol_1)
        print("n_mol_2 (ASP molecules):", n_mol_2)
        
        if end is None:
            end = trj_length
            
        # Initialize results list
        results = []
        for i, _ in enumerate(self.u.trajectory[start:end:skip]):
            if i % 10 == 0:
                print('Finished frame:', i)
            # Set positions and reshape
            pos_1 = ag_1.positions.reshape((n_mol_1, res_length_1, 3))
            pos_2 = ag_2.positions.reshape((n_mol_2, res_length_2, 3))
            # Compute pairwise contacts and store transposed results
            for p_1 in pos_1:
                for p_2 in pos_2:
                    results.append(self._pairwise_contacts(p_1, p_2, cutoff=cutoff).T)
                    
        # Aggregate results
        results = np.array(results)
        trj_mean = results.mean(axis=0)
        
        # Normalize contact map
        normalized_map = trj_mean / trj_mean.max()
        return normalized_map
    
    def save_contact_map(self, contact_map, output_file):
        """
        Save the contact map data to a .dat file with headers
        """
        # Create header with atom names
        header = '\t'.join(['AtomNames'] + [str(x) for x in self.xticks])
        
        # Open file and write data
        with open(output_file, 'w') as f:
            # Write header
            f.write(header + '\n')
            
            # Write data with row labels
            for i, row in enumerate(contact_map):
                row_data = '\t'.join([str(self.yticks[i])] + [f'{x:.6f}' for x in row])
                f.write(row_data + '\n')

if __name__ == '__main__':
    BASE_PATH = r"project3/"
    SYSTEM_NAME = "asp_trx"
    top = os.path.join(BASE_PATH, SYSTEM_NAME, "asp900ns.tpr")
    trj = os.path.join(BASE_PATH, SYSTEM_NAME, "merged_trajectory.xtc")
    
    m = ContactMap(top, trj)
    res = m.calculate_contacts(
        selection_1='resname TRITO and (name C* O*) and not (name C25 C26 C27 C28 C29 C30 C31 C32)',
        selection_2='resname ASP and not type H*',
        start=100, end=1000
    )
    
    # Save contact map data to .dat file
    m.save_contact_map(res, 'TRITO-ASP_contact_map.dat')
    
    # Create and save heatmap
    seaborn.heatmap(res, cmap='coolwarm', xticklabels=m.xticks, yticklabels=m.yticks)
    plt.savefig('TRITO-ASP_5H.jpg', dpi=300)
    plt.show()

In [ ]:
import os

import MDAnalysis as mda
import numpy as np
import seaborn
from MDAnalysis.analysis.distances import distance_array
from matplotlib import pyplot as plt
from numpy import vectorize
# from strip_water import strip_water


# from src.MoleculeSystem import MoleculeSystem

class ContactMap:

    def __init__(self, top_path: str, trj_path=None):
        self.ag = None
        if not trj_path:
            self.u = mda.Universe(top_path)
        else:
            self.u = mda.Universe(top_path, trj_path)

        self.yticks = None
        self.xticks = None

    @staticmethod
    @vectorize
    def binarize(x, cut_off: float):
        return 1 if x < cut_off else 0

    def _pairwise_contacts(self, a, b, cutoff):
        distances = distance_array(a, b)
        return self.binarize(distances, cutoff)

    def calculate_contacts(self, selection_1: str, selection_2: str, cutoff=5, start=0, end=None, skip=1):
        ag_1 = self.u.select_atoms(selection_1)
        ag_2 = self.u.select_atoms(selection_2)
        self.xticks = ag_1[ag_1.resids == ag_1.resids[0]].names
        self.yticks = ag_1[ag_1.resids == ag_1.resids[0]].names

        # Number of atoms in each residue
        res_length_1 = len(ag_1[ag_1.resids == ag_1.resids[0]])
        res_length_2 = len(ag_2[ag_2.resids == ag_2.resids[0]])

        # Number of molecules
        n_mol_1 = len(set(ag_1.resids))
        n_mol_2 = len(set(ag_2.resids))

        trj_length = len(self.u.trajectory)

        if end is None:
            end = trj_length

        results = []

        for i, _ in enumerate(self.u.trajectory[start:end:skip]):
            if i % 10 == 0:
                print('Finished frame:', i)
            # Set positions and reshape
            pos_1 = ag_1.positions.reshape((n_mol_1, res_length_1, 3))
            pos_2 = ag_2.positions.reshape((n_mol_2, res_length_2, 3))
            for p_1 in pos_1:
                for p_2 in pos_2:
                    results.append(self._pairwise_contacts(p_1, p_2, cutoff=cutoff))

        results = np.array(results)
        trj_mean = results.mean(axis=0)

        return trj_mean / trj_mean.max()
    def save_contact_map(self, data, output_file="contact_map.dat"):
        """
        Save the contact map to a .dat file in the format:
        Atom1 Atom2 InteractionStrength
        """
        with open(output_file, "w") as f:
            for i, atom1 in enumerate(self.xticks):
                for j, atom2 in enumerate(self.yticks):
                    f.write(f"{atom1} {atom2} {data[i, j]:.4f}\n")
        print(f"Contact map saved to {output_file}")


if __name__ == '__main__':
    BASE_PATH = r"project3/"
    SYSTEM_NAME = "aten_trx"
    top = os.path.join(BASE_PATH, SYSTEM_NAME, "aten900ns.tpr")
    trj = os.path.join(BASE_PATH, SYSTEM_NAME, "merged_aten.xtc")
    
    # strip_water(top, trj, water_resname='SOL')
    m = ContactMap(top, trj)
    #print(m.u.select_atoms('all').resnames)
    res = m.calculate_contacts(selection_1='resname ATEN and not type H*',
                               selection_2='resname ATEN and not type H*',
                               start=900, end=1000
                               )
    m.save_contact_map(res, output_file="ATEN_ATEN_contact_map.dat")


    seaborn.heatmap(res, cmap='coolwarm', xticklabels=m.xticks, yticklabels=m.yticks)
    plt.savefig('ATEN-ATEN_MH.jpg', dpi=300)
    plt.show()

In [ ]:
import os

import MDAnalysis as mda
import numpy as np
import seaborn
from MDAnalysis.analysis.distances import distance_array
from matplotlib import pyplot as plt
from numpy import vectorize
# from strip_water import strip_water


# from src.MoleculeSystem import MoleculeSystem

class ContactMap:

    def __init__(self, top_path: str, trj_path=None):
        self.ag = None
        if not trj_path:
            self.u = mda.Universe(top_path)
        else:
            self.u = mda.Universe(top_path, trj_path)

        self.yticks = None
        self.xticks = None

    @staticmethod
    @vectorize
    def binarize(x, cut_off: float):
        return 1 if x < cut_off else 0

    def _pairwise_contacts(self, a, b, cutoff):
        distances = distance_array(a, b)
        return self.binarize(distances, cutoff)

    def calculate_contacts(self, selection_1: str, selection_2: str, cutoff=5, start=0, end=None, skip=1):
        ag_1 = self.u.select_atoms(selection_1)
        ag_2 = self.u.select_atoms(selection_2)
        self.xticks = ag_1[ag_1.resids == ag_1.resids[0]].names
        self.yticks = ag_2[ag_2.resids == ag_2.resids[0]].names

        # Number of atoms in each residue
        res_length_1 = len(ag_1[ag_1.resids == ag_1.resids[0]])
        res_length_2 = len(ag_2[ag_2.resids == ag_2.resids[0]])

        # Number of molecules
        n_mol_1 = len(set(ag_1.resids))
        n_mol_2 = len(set(ag_2.resids))

        trj_length = len(self.u.trajectory)

        print("res_length_1 (TRITO):", res_length_1)  # Should be 37
        print("res_length_2 (ASP):", res_length_2)    # Should be 27
        print("n_mol_1 (TRITO molecules):", n_mol_1)
        print("n_mol_2 (ASP molecules):", n_mol_2)

        if end is None:
            end = trj_length

        # Initialize results list
        results = []

        for i, _ in enumerate(self.u.trajectory[start:end:skip]):
            if i % 10 == 0:
                print('Finished frame:', i)

            # Set positions and reshape
            pos_1 = ag_1.positions.reshape((n_mol_1, res_length_1, 3))
            pos_2 = ag_2.positions.reshape((n_mol_2, res_length_2, 3))

            # Compute pairwise contacts and store transposed results
            for p_1 in pos_1:
                for p_2 in pos_2:
                    results.append(self._pairwise_contacts(p_1, p_2, cutoff=cutoff).T)

        # Aggregate results
        results = np.array(results)
        trj_mean = results.mean(axis=0)

        # Normalize and return the contact map
        return trj_mean / trj_mean.max()


if __name__ == '__main__':
    BASE_PATH = r"project3/"
    SYSTEM_NAME = "aten_trx"
    top = os.path.join(BASE_PATH, SYSTEM_NAME, "aten900ns.tpr")
    trj = os.path.join(BASE_PATH, SYSTEM_NAME, "merged_aten.xtc")
    
    # strip_water(top, trj, water_resname='SOL')
    m = ContactMap(top, trj)
    #print(m.u.select_atoms('all').resnames)
    res = m.calculate_contacts(selection_1='resname TRITO and (name C* O*) and not (name C25 C26 C27 C28 C29 C30 C31 C32)',
                               selection_2='resname ATEN and not type H*',
                               start=900, end=1000
                               )

    seaborn.heatmap(res, cmap='coolwarm', xticklabels=m.xticks, yticklabels=m.yticks)
    plt.savefig('TRITO-ATEN_5H.jpg', dpi=300)
    plt.show()

In [ ]:
import os

import MDAnalysis as mda
import numpy as np
import seaborn
from MDAnalysis.analysis.distances import distance_array
from matplotlib import pyplot as plt
from numpy import vectorize
# from strip_water import strip_water


# from src.MoleculeSystem import MoleculeSystem

class ContactMap:

    def __init__(self, top_path: str, trj_path=None):
        self.ag = None
        if not trj_path:
            self.u = mda.Universe(top_path)
        else:
            self.u = mda.Universe(top_path, trj_path)

        self.yticks = None
        self.xticks = None

    @staticmethod
    @vectorize
    def binarize(x, cut_off: float):
        return 1 if x < cut_off else 0

    def _pairwise_contacts(self, a, b, cutoff):
        distances = distance_array(a, b)
        return self.binarize(distances, cutoff)

    def calculate_contacts(self, selection_1: str, selection_2: str, cutoff=5, start=0, end=None, skip=1):
        ag_1 = self.u.select_atoms(selection_1)
        ag_2 = self.u.select_atoms(selection_2)
        self.xticks = ag_1[ag_1.resids == ag_1.resids[0]].names
        self.yticks = ag_1[ag_1.resids == ag_1.resids[0]].names

        # Number of atoms in each residue
        res_length_1 = len(ag_1[ag_1.resids == ag_1.resids[0]])
        res_length_2 = len(ag_2[ag_2.resids == ag_2.resids[0]])

        # Number of molecules
        n_mol_1 = len(set(ag_1.resids))
        n_mol_2 = len(set(ag_2.resids))

        trj_length = len(self.u.trajectory)

        if end is None:
            end = trj_length

        results = []

        for i, _ in enumerate(self.u.trajectory[start:end:skip]):
            if i % 10 == 0:
                print('Finished frame:', i)
            # Set positions and reshape
            pos_1 = ag_1.positions.reshape((n_mol_1, res_length_1, 3))
            pos_2 = ag_2.positions.reshape((n_mol_2, res_length_2, 3))
            for p_1 in pos_1:
                for p_2 in pos_2:
                    results.append(self._pairwise_contacts(p_1, p_2, cutoff=cutoff))

        results = np.array(results)
        trj_mean = results.mean(axis=0)

        return trj_mean / trj_mean.max()
    def save_contact_map(self, data, output_file="contact_map.dat"):
        """
        Save the contact map to a .dat file in the format:
        Atom1 Atom2 InteractionStrength
        """
        with open(output_file, "w") as f:
            for i, atom1 in enumerate(self.xticks):
                for j, atom2 in enumerate(self.yticks):
                    f.write(f"{atom1} {atom2} {data[i, j]:.4f}\n")
        print(f"Contact map saved to {output_file}")


if __name__ == '__main__':
    BASE_PATH = r"project3/"
    SYSTEM_NAME = "fel_trx"
    top = os.path.join(BASE_PATH, SYSTEM_NAME, "fel900ns.tpr")
    trj = os.path.join(BASE_PATH, SYSTEM_NAME, "merged_fel.xtc")
    
    # strip_water(top, trj, water_resname='SOL')
    m = ContactMap(top, trj)
    #print(m.u.select_atoms('all').resnames)
    res = m.calculate_contacts(selection_1='resname FEL and not type H*',
                               selection_2='resname FEL and not type H*',
                               start=900, end=1000
                               )
    m.save_contact_map(res, output_file="FEL_FEL_contact_map.dat")

    seaborn.heatmap(res, cmap='coolwarm', xticklabels=m.xticks, yticklabels=m.yticks)
    plt.savefig('FEL-FEL_MH.jpg', dpi=300)
    plt.show()

In [ ]:
import os
import MDAnalysis as mda
import numpy as np
import seaborn
from MDAnalysis.analysis.distances import distance_array
from matplotlib import pyplot as plt
from numpy import vectorize

class ContactMap:
    def __init__(self, top_path: str, trj_path=None):
        self.ag = None
        if not trj_path:
            self.u = mda.Universe(top_path)
        else:
            self.u = mda.Universe(top_path, trj_path)
        self.yticks = None
        self.xticks = None
    
    @staticmethod
    @vectorize
    def binarize(x, cut_off: float):
        return 1 if x < cut_off else 0
    
    def _pairwise_contacts(self, a, b, cutoff):
        distances = distance_array(a, b)
        return self.binarize(distances, cutoff)
    
    def calculate_contacts(self, selection_1: str, selection_2: str, cutoff=5, start=0, end=None, skip=1):
        ag_1 = self.u.select_atoms(selection_1)
        ag_2 = self.u.select_atoms(selection_2)
        self.xticks = ag_1[ag_1.resids == ag_1.resids[0]].names
        self.yticks = ag_2[ag_2.resids == ag_2.resids[0]].names
        
        # Number of atoms in each residue
        res_length_1 = len(ag_1[ag_1.resids == ag_1.resids[0]])
        res_length_2 = len(ag_2[ag_2.resids == ag_2.resids[0]])
        
        # Number of molecules
        n_mol_1 = len(set(ag_1.resids))
        n_mol_2 = len(set(ag_2.resids))
        trj_length = len(self.u.trajectory)
        
        print("res_length_1 (TRITO):", res_length_1)
        print("res_length_2 (ASP):", res_length_2)
        print("n_mol_1 (TRITO molecules):", n_mol_1)
        print("n_mol_2 (ASP molecules):", n_mol_2)
        
        if end is None:
            end = trj_length
            
        # Initialize results list
        results = []
        for i, _ in enumerate(self.u.trajectory[start:end:skip]):
            if i % 10 == 0:
                print('Finished frame:', i)
            # Set positions and reshape
            pos_1 = ag_1.positions.reshape((n_mol_1, res_length_1, 3))
            pos_2 = ag_2.positions.reshape((n_mol_2, res_length_2, 3))
            # Compute pairwise contacts and store transposed results
            for p_1 in pos_1:
                for p_2 in pos_2:
                    results.append(self._pairwise_contacts(p_1, p_2, cutoff=cutoff).T)
                    
        # Aggregate results
        results = np.array(results)
        trj_mean = results.mean(axis=0)
        
        # Normalize contact map
        normalized_map = trj_mean / trj_mean.max()
        return normalized_map
    
    def save_contact_map(self, contact_map, output_file):
        """
        Save the contact map data to a .dat file with headers
        """
        # Create header with atom names
        header = '\t'.join(['AtomNames'] + [str(x) for x in self.xticks])
        
        # Open file and write data
        with open(output_file, 'w') as f:
            # Write header
            f.write(header + '\n')
            
            # Write data with row labels
            for i, row in enumerate(contact_map):
                row_data = '\t'.join([str(self.yticks[i])] + [f'{x:.6f}' for x in row])
                f.write(row_data + '\n')

if __name__ == '__main__':
    BASE_PATH = r"project3/"
    SYSTEM_NAME = "asp_trx"
    top = os.path.join(BASE_PATH, SYSTEM_NAME, "asp900ns.tpr")
    trj = os.path.join(BASE_PATH, SYSTEM_NAME, "merged_trajectory.xtc")
    
    m = ContactMap(top, trj)
    res = m.calculate_contacts(
        selection_1='resname TRITO and (name C* O*) and not (name C25 C26 C27 C28 C29 C30 C31 C32)',
        selection_2='resname ASP and not type H*',
        start=100, end=1000
    )
    
    # Save contact map data to .dat file
    m.save_contact_map(res, 'TRITO-ASP_contact_map.dat')
    
    # Create and save heatmap
    seaborn.heatmap(res, cmap='coolwarm', xticklabels=m.xticks, yticklabels=m.yticks)
    plt.savefig('TRITO-ASP_5H.jpg', dpi=300)
    plt.show()